In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def last(listo):
    return listo[len(listo)-1]

def differenceTable(listo):
    answer=[]
    for j in [1..len(listo)-1]:
        answer=answer+[listo[j]-listo[j-1]]
    return answer
    

In [26]:
print differenceTable([1,5,3])

[4, -2]


In [ ]:
data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()# output snipped 

import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
# output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

121
[0, 24*x^3 + 32*x]


In [4]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print poly

24*x^3 + 32*x
276*x^6 - 32*x^4 - 192*x^2
2048*x^9 - 237568/27*x^7 + 32768/27*x^5 + 131072/27*x^3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
49152*x^15 - 1072627712/1125*x^13 + 4173856768/675*x^11 - 45736787968/3375*x^9 + 1564475392/675*x^7 + 6257901568/1125*x^5
184024*x^18 - 144266176/27*x^16 + 43337949824/729*x^14 - 217405085696/729*x^12 + 140124878848/243*x^10 - 77900890112/729*x^8 - 155801780224/729*x^6


In [5]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print polynomialnumerator(D(poly))

24*x^3 + 32*x
276*x^6 - 32*x^4 - 192*x^2
55296*x^9 - 237568*x^7 + 32768*x^5 + 131072*x^3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
165888000*x^15 - 3217883136*x^13 + 20869283840*x^11 - 45736787968*x^9 + 7822376960*x^7 + 18773704704*x^5
134153496*x^18 - 3895186752*x^16 + 43337949824*x^14 - 217405085696*x^12 + 420374636544*x^10 - 77900890112*x^8 - 155801780224*x^6


In [2]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==0:data=data+[index]
print data

[]


In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd>0:data=data+[sd]
print Set(data)

{1, 2, 3, 4, 6, 8, 10, 16, 20}


In [4]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=0
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)
        


(0, 0, 2, 3)
(11, 1, 16, 36)
(22, 2, 16, 67)
(33, 3, 16, 98)
(44, 4, 16, 129)
(55, 5, 16, 160)
(66, 6, 16, 191)
(77, 7, 16, 222)
(88, 8, 16, 253)
(99, 9, 16, 284)
(110, 10, 16, 315)
[33, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [5]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=1
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(1, 0, 4, 6)
(12, 1, 4, 37)
(23, 2, 4, 68)
(34, 3, 4, 99)
(45, 4, 4, 130)
(56, 5, 4, 161)
(67, 6, 4, 192)
(78, 7, 4, 223)
(89, 8, 4, 254)
(100, 9, 4, 285)
(111, 10, 4, 316)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [6]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=2
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(2, 0, 4, 9)
(13, 1, 4, 40)
(24, 2, 4, 71)
(35, 3, 4, 102)
(46, 4, 4, 133)
(57, 5, 4, 164)
(68, 6, 4, 195)
(79, 7, 4, 226)
(90, 8, 4, 257)
(101, 9, 4, 288)
(112, 10, 4, 319)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [7]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(3, 0, 6, 12)
(14, 1, 6, 43)
(25, 2, 6, 74)
(36, 3, 6, 105)
(47, 4, 6, 136)
(58, 5, 6, 167)
(69, 6, 6, 198)
(80, 7, 6, 229)
(91, 8, 6, 260)
(102, 9, 6, 291)
(113, 10, 6, 322)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [8]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=4
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(4, 0, 4, 15)
(15, 1, 4, 46)
(26, 2, 4, 77)
(37, 3, 4, 108)
(48, 4, 4, 139)
(59, 5, 4, 170)
(70, 6, 4, 201)
(81, 7, 4, 232)
(92, 8, 4, 263)
(103, 9, 4, 294)
(114, 10, 4, 325)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [9]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=5
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(5, 0, 8, 18)
(16, 1, 8, 49)
(27, 2, 8, 80)
(38, 3, 8, 111)
(49, 4, 8, 142)
(60, 5, 8, 173)
(71, 6, 8, 204)
(82, 7, 8, 235)
(93, 8, 8, 266)
(104, 9, 8, 297)
(115, 10, 8, 328)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [10]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=6
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(6, 0, 3, 21)
(17, 1, 3, 52)
(28, 2, 3, 83)
(39, 3, 3, 114)
(50, 4, 3, 145)
(61, 5, 3, 176)
(72, 6, 3, 207)
(83, 7, 3, 238)
(94, 8, 3, 269)
(105, 9, 3, 300)
(116, 10, 3, 331)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [11]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=7
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(7, 0, 20, 24)
(18, 1, 20, 55)
(29, 2, 20, 86)
(40, 3, 20, 117)
(51, 4, 20, 148)
(62, 5, 20, 179)
(73, 6, 20, 210)
(84, 7, 20, 241)
(95, 8, 20, 272)
(106, 9, 20, 303)
(117, 10, 20, 334)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [12]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=8
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(8, 0, 10, 27)
(19, 1, 10, 58)
(30, 2, 10, 89)
(41, 3, 10, 120)
(52, 4, 10, 151)
(63, 5, 10, 182)
(74, 6, 10, 213)
(85, 7, 10, 244)
(96, 8, 10, 275)
(107, 9, 10, 306)
(118, 10, 10, 337)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [13]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=9
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(9, 0, 6, 30)
(20, 1, 6, 61)
(31, 2, 6, 92)
(42, 3, 6, 123)
(53, 4, 6, 154)
(64, 5, 6, 185)
(75, 6, 6, 216)
(86, 7, 6, 247)
(97, 8, 6, 278)
(108, 9, 6, 309)
(119, 10, 6, 340)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [14]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=11
residue=10
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(-1, -1, 1, 0)
(10, 0, 1, 31)
(21, 1, 1, 62)
(32, 2, 1, 93)
(43, 3, 1, 124)
(54, 4, 1, 155)
(65, 5, 1, 186)
(76, 6, 1, 217)
(87, 7, 1, 248)
(98, 8, 1, 279)
(109, 9, 1, 310)
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]


In [15]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=0
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)
        



(0, 0, 1, 3)
(13, 1, 10, 42)
(26, 2, 10, 79)
(39, 3, 10, 116)
(52, 4, 10, 153)
(65, 5, 10, 190)
(78, 6, 10, 227)
(91, 7, 10, 264)
(104, 8, 10, 301)
(117, 9, 10, 338)
[39, 37, 37, 37, 37, 37, 37, 37, 37]


In [19]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=1
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)
        




(1, 0, 1, 6)
(14, 1, 1, 43)
(27, 2, 1, 80)
(40, 3, 1, 117)
(53, 4, 1, 154)
(66, 5, 1, 191)
(79, 6, 1, 228)
(92, 7, 1, 265)
(105, 8, 1, 302)
(118, 9, 1, 339)
[37, 37, 37, 37, 37, 37, 37, 37, 37]


In [20]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=2
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(2, 0, 2, 9)
(15, 1, 2, 46)
(28, 2, 2, 83)
(41, 3, 2, 120)
(54, 4, 2, 157)
(67, 5, 2, 194)
(80, 6, 2, 231)
(93, 7, 2, 268)
(106, 8, 2, 305)
(119, 9, 2, 342)
[37, 37, 37, 37, 37, 37, 37, 37, 37]


In [21]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(3, 0, 6, 12)
(16, 1, 6, 49)
(29, 2, 6, 86)
(42, 3, 6, 123)
(55, 4, 6, 160)
(68, 5, 6, 197)
(81, 6, 6, 234)
(94, 7, 6, 271)
(107, 8, 6, 308)
[37, 37, 37, 37, 37, 37, 37, 37]


In [22]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=4
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(4, 0, 4, 15)
(17, 1, 4, 52)
(30, 2, 4, 89)
(43, 3, 4, 126)
(56, 4, 4, 163)
(69, 5, 4, 200)
(82, 6, 4, 237)
(95, 7, 4, 274)
(108, 8, 4, 311)
[37, 37, 37, 37, 37, 37, 37, 37]


In [23]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=5
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(5, 0, 6, 18)
(18, 1, 6, 55)
(31, 2, 6, 92)
(44, 3, 6, 129)
(57, 4, 6, 166)
(70, 5, 6, 203)
(83, 6, 6, 240)
(96, 7, 6, 277)
(109, 8, 6, 314)
[37, 37, 37, 37, 37, 37, 37, 37]


In [24]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=6
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(6, 0, 10, 21)
(19, 1, 10, 58)
(32, 2, 10, 95)
(45, 3, 10, 132)
(58, 4, 10, 169)
(71, 5, 10, 206)
(84, 6, 10, 243)
(97, 7, 10, 280)
(110, 8, 10, 317)
[37, 37, 37, 37, 37, 37, 37, 37]


In [25]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=7
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(7, 0, 12, 24)
(20, 1, 12, 61)
(33, 2, 12, 98)
(46, 3, 12, 135)
(59, 4, 12, 172)
(72, 5, 12, 209)
(85, 6, 12, 246)
(98, 7, 12, 283)
(111, 8, 12, 320)
[37, 37, 37, 37, 37, 37, 37, 37]


In [26]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=8
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(8, 0, 12, 27)
(21, 1, 12, 64)
(34, 2, 12, 101)
(47, 3, 12, 138)
(60, 4, 12, 175)
(73, 5, 12, 212)
(86, 6, 12, 249)
(99, 7, 12, 286)
(112, 8, 12, 323)
[37, 37, 37, 37, 37, 37, 37, 37]


In [27]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=9
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(9, 0, 20, 30)
(22, 1, 20, 67)
(35, 2, 20, 104)
(48, 3, 20, 141)
(61, 4, 20, 178)
(74, 5, 20, 215)
(87, 6, 20, 252)
(100, 7, 20, 289)
(113, 8, 20, 326)
[37, 37, 37, 37, 37, 37, 37, 37]


In [28]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=10
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(10, 0, 18, 33)
(23, 1, 18, 70)
(36, 2, 18, 107)
(49, 3, 18, 144)
(62, 4, 18, 181)
(75, 5, 18, 218)
(88, 6, 18, 255)
(101, 7, 18, 292)
(114, 8, 18, 329)
[37, 37, 37, 37, 37, 37, 37, 37]


In [29]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=11
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(11, 0, 8, 36)
(24, 1, 8, 73)
(37, 2, 8, 110)
(50, 3, 8, 147)
(63, 4, 8, 184)
(76, 5, 8, 221)
(89, 6, 8, 258)
(102, 7, 8, 295)
(115, 8, 8, 332)
[37, 37, 37, 37, 37, 37, 37, 37]


In [30]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=12
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(-1, -1, 1, 0)
(12, 0, 1, 37)
(25, 1, 1, 74)
(38, 2, 1, 111)
(51, 3, 1, 148)
(64, 4, 1, 185)
(77, 5, 1, 222)
(90, 6, 1, 259)
(103, 7, 1, 296)
(116, 8, 1, 333)
[37, 37, 37, 37, 37, 37, 37, 37, 37]


In [16]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=17
residue=0
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(0, 0, 2, 3)
(17, 1, 84, 54)
(34, 2, 84, 103)
(51, 3, 84, 152)
(68, 4, 84, 201)
(85, 5, 84, 250)
(102, 6, 84, 299)
(119, 7, 84, 348)
[51, 49, 49, 49, 49, 49, 49]


In [ ]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=13
residue=5
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

In [17]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=19
residue=0
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(0, 0, 1, 3)
(19, 1, 14, 60)
(38, 2, 14, 115)
(57, 3, 14, 170)
(76, 4, 14, 225)
(95, 5, 14, 280)
(114, 6, 14, 335)
[57, 55, 55, 55, 55, 55]


In [18]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=23
residue=0
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(0, 0, 2, 3)


/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python2.7/site-packages/sage/rings/algebraic_closure_finite_field.py:1122: RuntimeWarning: cypari2 leaked 198112 bytes on the PARI stack
  return self._pseudo_conway_lattice.polynomial(n)


KeyboardInterrupt: 